# Understanding Object Oriented Languages by Building One

Let’s try to build an embedded programming language to play with (and hopefully understand a thing or two about) object-orientation.

## Showing my hands

In order to make the code more readable, I introduce a small set of helpers to leverage our host language (Python here), yet make the result very readable.

In [1]:
# Syntactic sugar: access elements of the dictionary with the same syntax than attributes
class MagicDict:
    def __init__(self, d):
        self._d = d
    def __getattr__(self, attr):
        if attr == "_d":
            super().__getattr__(attr)
            return
        return self._d[attr]
    def __setattr__(self, attr, value):
        if attr == "_d":
            super().__setattr__(attr, value)
            return
        if attr not in self._d:
            raise Exception("No such key " + attr)
        self._d[attr] = value
    def __str__(self):
        return str(self._d)

## Misconception 1: Classes as Namespaces

Let’s create a simple logging package, with a Log class, and a few static methods to define multiple levels of verbosity.

We now have a pretty (ish) way of running code, namely `log.info`, but is this any different than using a prefix (e.g. `log_info`)?

In [2]:
class Log:
    "A class misused as namespace"
    @staticmethod
    def info(message):
        print("INFO", message)
    @staticmethod
    def error(message):
        print("ERROR", message)

def log_info(message):
    print("INFO", message)
def log_error(message):
    print("ERROR", message)
    
Log.info("a class is not a namespace!")
log_info("a prefix would do just as well…")

INFO a class is not a namespace!
INFO a prefix would do just as well…


Sometimes, classes are created to be simple “bags” of information – bundling together different datum. While the problem might require exactly this approach, be aware that this introduces a lot of boilerplate code (repetitive, and adding very little value).

A better alternative, if your language allows it, is to use a plain and simple dictionary.

In [4]:
class Person:
    "A bag of values"
    def __init__(self, name, age):
        self._name = name
        self._age = age
    def setName(self, name):
        self._name = name
    def getName(self):
        return self._name
    def setAge(self, age):
        self._age = age
    def getAge(self):
        return self._age
    def __str__(self):
        return str({"name": self._name, "age": self._age})
    
John = Person("John Doe", 21)
John.setAge(23)
print(John)

# note that I do not need to define anything.
Jack = {"name": "Jack the Ripper", "age": 21}
Jack["age"] = 23
print(Jack)

{'name': 'John Doe', 'age': 23}
{'name': 'Jack the Ripper', 'age': 23}


## Misconception 2: Object-Orientation Deals with Information Hiding (aka Encapsulation)

Information hiding has multiple meanings, so let’s get one out of the way: in "Enterprise" programming, team jealously keep tight control over their code and limit interactions with other pieces of the program. Much could be said about this attitude, and how it relates to the functioning of an organisation, but there is also an acute, technical problem: when sharing memory, there is simply no boundary that can be enforced — therefore, those `private` keywords are just documentation (sometimes enforced by the compiler, but easy to defeat at runtime).

Another, more interesting, feature of information hiding is to provide abstraction (meaning users can work at a higher level, and do not need to keep in mind at all times the lower-levels details). Let us look at the (archetypical) example of the counter: a value that can only be incremented (the `inc` method), and inspected (the `count` one). I’ll implement this by using a closure over a number:

In [8]:
def Counter():
    "A closure encapsulate our state"
    count = 0
    def inc():
        nonlocal count
        count += 1
    def pcount():
        return count
    return MagicDict({
        "inc": inc,
        "count": pcount,
    })

c1 = Counter()
c1.inc()
c1.count()

1

Now, for fun and profit, let’s augment the closure by also implementing the way to decrease the value (the `decr` method).

In [9]:
def Gauge():
    count = 0
    def inc():
        nonlocal count
        count += 1
    def decr():
        nonlocal count
        if count > 0:
            count -= 1
    def pcount():
        return count
    return MagicDict({
        "inc": inc,
        "count": pcount,
        "decr": decr,
    })

def MyServer(counter):
    "starts a new server, and count the requests"
    counter.inc()
    
c = Counter()
MyServer(c)
print("Counter", c.count())

g = Gauge()
MyServer(g)
g.decr()
print("Gauge", g.count())

Counter 1
Gauge 0


Note the call to `inc`, _independent_ of the underlying closure / implementation behind it; so that the `MyServer` function is able to use *both* a gauge and a counter.

Our implementation uses something called “dynamic dispatch”, meaning that, at runtime, the dictionary returned by either the `Gauge` or `Counter` function is iterated until the right method is found (by matching its name), then executed. Other languages will be able to detect at compile time which code path to dispatch to (called “static dispatch”), and some will be able to accelerate dynamic dispatch between calls (“inline caching”).

## Lets Start Baking

Now comes the real fun: building our own dispatch system, and therefore our object-oriented mini language. Say hello to Cookie!

We will use facilities of the Python programing language to make our life easier: ours is an “embedded” programming language, but note that nothing in what we use depends on having an object-oriented language to start from (i.e. there is no magic).

Cookie will allow to define new type of cookies, and execute functions on said cookies by using the `do` method. 

In [11]:
class Cookie:
    """Cookies are delicious mix of fat and sugar.

    We can do many things with them, so define your own and what you like to `do` with your cookies.
    """
    def __str__(self):
        return self._name
    
    def __init__(self, name, operations):
        "init creates a new cookie type, with associated operations"
        self._name = name
        self._operations = operations
        
    def do(self, op, *args):
        "do execute the requested operation, possibly with arguments"
        if not op in self._operations:
            raise Exception("no such operation", op)
        return self._operations[op](*args)

# very simple method, returning a constant value
def bake_butter():
    "I like to bake at a constant temperature"
    return "cooking at 100°"

# also simple method, but the value depends on the input.
def bake_chocolate(color):
    "I like to do things differently, based on the type of chocolate"
    if color == "white":
        return "cooking at 90°"
    else:
        return "cooking at 80°"

c1 = Cookie("ButterCookie", {"bake": bake_butter})
c2 = Cookie("ChocolateCookie", {"bake": bake_chocolate})
print(f'{c1}:{c1.do("bake")}')
print(f'{c2}:{c2.do("bake", "white")}')

ButterCookie:cooking at 100°
ChocolateCookie:cooking at 90°


But, wait! This is cheating: we currently have only methods that act on their parameters (static methods). How do we also attach values to it (like we did in our counter example)?

Lets add values to the cookie at construction time, and make an automated setter and getter since we are at it (in case you did not notice, I am not a big fan of having to define those manually).

In [13]:
class Cookie2:
    def __str__(self):
        return self._name
    
    def __init__(self, name, operations, values):
        self._name = name
        self._values = values
        self._operations = operations
        
    def do(self, op, *args):
        "Do executes the operation. If it starts with setXXX / getXXX, it automatically accesses the value with the name"
        if op.startswith("set"):
            self._values[op[3:].lower()] = args[0]
        elif op.startswith("get"):
            return self._values[op[3:].lower()]
        else:
            if not op in self._operations:
                raise Exception("no such operation", op)
            return self._operations[op](self, *args)
        
def bake_nuts(self):
    return "baking " + str(self.do("getWeight") * 12) + " minutes"

nCookie = Cookie2("Nuts cookie", {"bake": bake_nuts}, {"weight": 0.2})
print(nCookie.do("bake"))
nCookie.do("setWeight", 0.3)
print(nCookie.do("bake"))

baking 2.4000000000000004 minutes
baking 3.5999999999999996 minutes


Cooking one cookie at a time is OK, but a bit … limited, right? (OK, Javascript stops here, but why should we?) Let's make many cookies at a time, using the tool for the job.

** For those (youngsters) of you who did not get the joke, Javascript initially provided only a form of object-orientation called “prototype”, where objects are the only concepts (no classes). Those languages have mostly disappeared since they are really, really hard to optimize – classes make it much easier to share code, minimizing the impact on memory, cache friendliness, …

In [17]:
class cookie:
    """cookie is a helper class to give us the do/set/get methods
    
    Note the trick: a cookie always defers to the cuttter for the actual implementation of the methods, but has local values"""
    def __str__(self):
        return self._cutter.printCookie()

    def __init__(self, cutter, attr):
        self._cutter = cutter
        self._values = attr
        
    def do(self, op, *args):
        if op.startswith("set"):
            self._values[op[3:].lower()] = args[0]
        elif op.startswith("get"):
            return self._values[op[3:].lower()]
        else:
            return self._cutter.do(self, op)

class CookieCutter:
    def __str__(self):
        return self._name + " cutter"
    
    def __init__(self, name, operations):
        self._name = name
        self._operations = operations
        
    def do(self, cookie, op, *args):
        if not op in self._operations:
            raise Exception("no such operation", op)
        return self._operations[op](cookie, *args)
    
    def printCookie(self):
        return "I am a " + self._name + " cookie"
    
    def newCookie(self, attr):
        c = cookie(self, attr)
        return c

def bake_macadamia(cookie):
    if cookie.do("getColor") == "white":
        return str(30)
    else:
        return str(100)

macadamiaCookieCutter = CookieCutter("MacadamiaCookie", {"bake": bake_macadamia})
noonCookie = macadamiaCookieCutter.newCookie({"color": "white"})
midnightCookie = macadamiaCookieCutter.newCookie({"color": "dark"})

print(f"{noonCookie}, with {noonCookie.do('getColor')} chocolate, baked at {noonCookie.do('bake')}")
print(f"{midnightCookie}, with {midnightCookie.do('getColor')} chocolate, baked at {midnightCookie.do('bake')}")

I am a MacadamiaCookie cookie, with white chocolate, baked at 30
I am a MacadamiaCookie cookie, with dark chocolate, baked at 100


## Implementing inheritance

Taking further the concept of code (or rather, cookie) re-use, we can extend the cutters so that they can also enlist the help of other cutters, who might know how to do some jobs.

Those cutters might, in turn rely on some methods from their own ancestors cutters, or indeed their children ones (typically called “abstract” methods in static languages.)

In [23]:
class CookieCutter:
    def __str__(self):
        return self._name + " cutter"
    
    def __init__(self, name, operations, helpers):
        self._name = name
        self._operations = operations
        self._helpers = helpers
        
    def do(self, cookie, op, *args):
        if not op in self._operations:
            for helper in self._helpers:
                try:
                    return helper.do(cookie, op, *args)
                except:
                    pass
            raise Exception("no such operation", op)
        return self._operations[op](cookie, *args)
    
    def printCookie(self):
        return self._name + " cookie"
    
    def newCookie(self, attr):
        c = cookie(self, attr)
        return c

def contemplate(cookie):
    return f"I am looking at a delicious {cookie}, baked at {cookie.do('bake')}. Temptation is strong."

nutsCookieCutter = CookieCutter("NutsCookie", {"contemplate": contemplate}, [])
macadamiaCookieCutter = CookieCutter("MacadamiaCookie", {"bake": bake_macadamia}, [nutsCookieCutter])
noonCookie = macadamiaCookieCutter.newCookie({"color": "white"})

print(noonCookie.do('contemplate'))

I am looking at a delicious MacadamiaCookie cookie, baked at 30. Temptation is strong.


### Exercise:
What happens when multiple parents (or children) implement the same operation?

# Parametric dispatch

We had some fun having the code jump around between cutters, both up and down the hierarchy. But it is also possible to chose where to dispatch code based on the value of the arguments. This is not commonly encountered at the language-level (you have to manually encode it), and renders most optimizations possible. But the form is fairly useful when implementing domain-specific dispatch.